In [66]:
# %%
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install scikit-learn
%pip install matplotlib
%pip install timm


Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip




     ---------------------------------------- 0.0/47.5 kB ? eta -:--:--
     -------- ------------------------------- 10.2/47.5 kB ? eta -:--:--
     ---------------- --------------------- 20.5/47.5 kB 165.2 kB/s eta 0:00:01
     ---------------- --------------------- 20.5/47.5 kB 165.2 kB/s eta 0:00:01
     ---------------- --------------------- 20.5/47.5 kB 165.2 kB/s eta 0:00:01
     ---------------- --------------------- 20.5/47.5 kB 165.2 kB/s eta 0:00:01
     ------------------------- ------------- 30.7/47.5 kB 87.5 kB/s eta 0:00:01
     ------------------------- ------------- 30.7/47.5 kB 87.5 kB/s eta 0:00:01
     --------------------------------- ----- 41.0/47.5 kB 98.5 kB/s eta 0:00:01
     -------------------------------------- 47.5/47.5 kB 108.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
    --


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [67]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
import timm
import numpy as np

In [68]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [69]:
# Define dataset paths
train_data_path = 'C:\\projects\\BloodNets\\blood_data\\train'
test_data_path = 'C:\\projects\\BloodNets\\blood_data\\test'

# Load datasets
train_data = datasets.ImageFolder(train_data_path, transform=transform)
test_data = datasets.ImageFolder(test_data_path, transform=transform)


In [70]:
# Training function
def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * inputs.size(0)
    return total_loss / len(loader.dataset)

In [71]:
# Testing function
def test(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    metrics = {
        'loss': total_loss / len(loader.dataset),
        'accuracy': accuracy_score(all_labels, all_preds),
        'precision': precision_score(all_labels, all_preds, average='weighted'),
        'recall': recall_score(all_labels, all_preds, average='weighted'),
        'f1_score': f1_score(all_labels, all_preds, average='weighted')
    }
    return metrics

In [72]:
# Initialize K-Fold cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True)



In [73]:
# Initialize EfficientNetV2 model
model = timm.create_model('efficientnetv2_rw_m', pretrained=True, num_classes=len(train_data.classes))
model = model.to(device)

model.safetensors:   0%|          | 0.00/214M [00:00<?, ?B/s]

In [74]:
# Define criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [75]:
# Main cross-validation loop
for fold, (train_idx, test_idx) in enumerate(kf.split(train_data)):
    print(f"Fold {fold + 1}/{k_folds}")

    # Create data loaders for training and testing
    train_loader = torch.utils.data.DataLoader(
        train_data, batch_size=32, sampler=torch.utils.data.SubsetRandomSampler(train_idx))
    test_loader = torch.utils.data.DataLoader(
        train_data, batch_size=32, sampler=torch.utils.data.SubsetRandomSampler(test_idx))

Fold 1/5
Fold 2/5
Fold 3/5
Fold 4/5
Fold 5/5


In [77]:
 # Train and test for each fold
for epoch in range(10):  # Adjust number of epochs as needed
        train_loss = train(model, train_loader, criterion, optimizer, device)
        metrics = test(model, test_loader, criterion, device)

        # Print metrics
        print(f"Epoch {epoch + 1}/10")
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Test Loss: {metrics['loss']:.4f}")
        print(f"Test Accuracy: {metrics['accuracy']*100:.4f}")
        print(f"Test Precision: {metrics['precision']*100:.4f}")
        print(f"Test Recall: {metrics['recall']*100:.4f}")
        print(f"Test F1 Score: {metrics['f1_score']*100:.4f}")

    # Save model for each fold if needed
torch.save(model.state_dict(), f'efficientnetv2_fold_{fold + 1}.pth')

KeyboardInterrupt: 